<a href="https://colab.research.google.com/github/sudoghut/chinese-book-classification/blob/main/chinese_book_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tqdm
!pip install -U sentence-transformers
!pip install -U xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.0 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=0f6705864e022eb23980ac3ca49a94249a8e83abd1f8ca56ce628e65b08f044e
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
import torch
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
import pandas as pd
import xgboost as xgb
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import os
import datetime

In [3]:
with open('input.txt', 'r') as f:
  titles = [title.strip() for title in f.readlines()]
titles[:5]

['进修堂文集诗集', '马书奎稿', '游莱三体诗稿', '鲍觉生诗钞', '赋则']

In [4]:
if os.path.exists('/content/model.bin'):
  print('Model already exists. Skipping download.')
else:
  !wget https://raw.githubusercontent.com/sudoghut/chinese-book-classification/main/model.bin

if os.path.exists('/content/titles_all.txt'):
  print('titles_all already exists. Skipping download.')
else:
  !wget https://raw.githubusercontent.com/sudoghut/chinese-book-classification/main/titles_all.txt

if os.path.exists('/content/training.xlsx'):
  print('training.xlsx already exists. Skipping download.')
else:
  !wget https://raw.githubusercontent.com/sudoghut/chinese-book-classification/main/training.xlsx

if os.path.exists('/content/book_category_match.xlsx'):
  print('book_category_match.xlsx already exists. Skipping download.')
else:
  !wget https://raw.githubusercontent.com/sudoghut/chinese-book-classification/main/book_category_match.xlsx


--2024-01-20 23:58:47--  https://raw.githubusercontent.com/sudoghut/chinese-book-classification/main/model.bin
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3261903 (3.1M) [application/octet-stream]
Saving to: ‘model.bin’

model.bin           100%[===================>]   3.11M  --.-KB/s    in 0.01s   

2024-01-20 23:58:47 (217 MB/s) - ‘model.bin’ saved [3261903/3261903]

--2024-01-20 23:58:47--  https://raw.githubusercontent.com/sudoghut/chinese-book-classification/main/titles_all.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Leng

In [5]:
prepared_title_list = [line.rstrip('\n') for line in open('/content/titles_all.txt')]
title_all = titles + prepared_title_list

In [6]:
def cosine_similarity(embedding1, embedding2):
    return torch.nn.functional.cosine_similarity(embedding1, embedding2, dim=0)

top_number_setting = 50
def similarity_calculation(model):
    top_similarities_list = []
    total = len(titles) * len(prepared_title_list)

    with tqdm(total=total, desc="Calculating Similarities") as pbar:
        embeddings = model.encode(title_all)
        for i in range(len(titles)):
            similarities = []
            for j in range(len(titles), len(title_all)):
                similarity = cosine_similarity(torch.tensor(embeddings[i]), torch.tensor(embeddings[j]))
                try:
                    similarities.append(((titles[i], prepared_title_list[j-len(titles)]), similarity.item()))
                except:
                    print(i)
                    print(j)
                    raise
                pbar.update(1)
            # print(len(similarities))
            sorted_similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
            if len(sorted_similarities) < top_number_setting:
                top_similarities = sorted_similarities
            else:
                top_similarities = sorted(similarities, key=lambda x: x[1], reverse=True)[:top_number_setting]
            # print(top_similarities[:5])
            # for pair, sim in top_similarities:
            #     print(f"Cosine similarity between \"{pair[0]}\" and \"{pair[1]}\": {sim}")
            top_similarities_list.append(top_similarities)
    top_similarities_list = [[j[0][0], j[0][1], "{:.4f}".format(j[1])] for i in top_similarities_list for j in i]

    return top_similarities_list

In [7]:
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
top_similarities_list = similarity_calculation(model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/968 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.09k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

unigram.json:   0%|          | 0.00/14.8M [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

Calculating Similarities: 100%|██████████| 13292352/13292352 [09:49<00:00, 22538.10it/s]


In [8]:
df = pd.DataFrame(top_similarities_list, columns=['title_1', 'title_2', 'similarity'])
print(df.shape)
df[:5]

(75200, 3)


,title_1,title_2,similarity
0,进修堂文集诗集,敬业堂诗集,0.9106
1,进修堂文集诗集,岁寒堂诗话,0.8700
2,进修堂文集诗集,二老堂诗话,0.8686
3,进修堂文集诗集,娱书堂诗话,0.8677
4,进修堂文集诗集,嵗寒堂诗话,0.8669


In [9]:
model = xgb.XGBClassifier()
model.load_model("/content/model.bin")
df_train_for_vectorizer = pd.read_excel("/content/training.xlsx")

combined_data = pd.concat(
    [df_train_for_vectorizer["x_1"], df_train_for_vectorizer["x_2"]]
)
vectorizer = TfidfVectorizer(analyzer="char")
vectorizer.fit(combined_data)
X_1 = vectorizer.transform(df["title_1"])
X_2 = vectorizer.transform(df["title_2"])
X = np.hstack((X_1.toarray(), X_2.toarray()))
print(X.shape)
predicted_y = model.predict(X)
df["predicted_y"] = predicted_y
df_list = df.values.tolist()
# df.to_excel("compare_categories_all.xlsx", index=False)
compare_categories_summarized = []
count = 0
match_flag = False
previous_row = df_list[0]
for row in df_list:
    # First record got predicted_y = 1
    if row[3] == 1 and row[0] != previous_row[0]:
        compare_categories_summarized.append(row + ["first predict"])
        previous_row = row
        match_flag = True
        count += 1
    # Within the same title match, the first record got predicted_y = 1
    elif (
        row[3] == 1
        and row[0] == previous_row[0]
        and previous_row[3] == 0
        and match_flag == False
    ):
        compare_categories_summarized.append(row + ["inner predict"])
        previous_row = row
        if count == (top_number_setting - 1):
            count = 0
        else:
            count += 1
            match_flag = True
    # At the last record. Can't find any predicted_y = 1 within the same title match
    elif count == (top_number_setting - 1):
        if match_flag == False:
            compare_categories_summarized.append([row[0], "", "", 0, "no match"])
        previous_row = row
        count = 0
        match_flag = False
    else:
        count += 1
        previous_row = row
# export compare_categories_summarized to excel
df_summarized = pd.DataFrame(
    compare_categories_summarized,
    columns=["title_1", "title_2", "similarity", "predicted_y", "reason"],
)
# df_summarized.to_excel("compare_categories_summarized.xlsx", index=False)

(75200, 5618)


In [10]:
title_1_with_zero_prediction = df_summarized[df_summarized["predicted_y"] == 0]["title_1"]
df_summarized_no_predicted_y = df[df["title_1"].isin(title_1_with_zero_prediction)]
df_summarized_no_predicted_y.head()

,title_1,title_2,similarity,predicted_y
200,赋则,琴赋,0.9541,0
201,赋则,罗织经,0.9438,0
202,赋则,权书,0.9427,0
203,赋则,皮门,0.9381,0
204,赋则,胁门,0.9367,0


In [11]:
def calculate_similarity(title1, title2):
    title1 = "".join(set(title1))
    title2 = "".join(set(title2))
    similarity = 0

    for i in range(len(title1)):
        if title1[i] in title2:
            similarity += 1
    return similarity

# Apply the function to calculate similarity for each pair of titles
df_summarized_no_predicted_y["similarity"] = df_summarized_no_predicted_y.apply(
    lambda row: calculate_similarity(row["title_1"], row["title_2"]), axis=1
)

# # Function to select the top 'n' similar pairs for each title1 and export to a new CSV
def export_top_similar_pairs(df_summarized_no_predicted_y, n):
    # Sort by similarity in descending order
    df_summarized_no_predicted_y = df_summarized_no_predicted_y.sort_values(by="similarity", ascending=False)

    # Group by title_1 and select top 'n' for each title_1
    top_pairs = df_summarized_no_predicted_y.groupby("title_1").head(n)

    # Skip the records where similarity is 0
    top_pairs = top_pairs[top_pairs["similarity"] != 0]

    return top_pairs




simple_similarity_top_pairs = export_top_similar_pairs(df_summarized_no_predicted_y, 1)
# simple_similarity_top_pairs.to_excel("simple_similarity_top_pairs.xlsx", index=False)
simple_similarity_top_pairs.head()

<ipython-input-11-945898474c2f>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_summarized_no_predicted_y["similarity"] = df_summarized_no_predicted_y.apply(


,title_1,title_2,similarity,predicted_y
13600,两晋南北史乐府,拟两晋南北史乐府,7,0
36266,山井鼎七经孟子考文,七经孟子考文补遗,6,0
58800,李义山文集注,李义山文集笺注,6,0
16850,文心雕龙辑注,文心雕龙辑注,6,0
21550,海陵从政录,海陵从政录,5,0


In [12]:
for i, row in df_summarized.iterrows():
    if row["predicted_y"] == 0:
        try:
            df_summarized["title_2"][i] = simple_similarity_top_pairs[simple_similarity_top_pairs["title_1"] == row["title_1"]]["title_2"].values[0]
            df_summarized["predicted_y"][i] = 1
            df_summarized["reason"][i] = "simple similarity"
        except:
            df_summarized["title_2"][i] = ""

df_summarized.head()


<ipython-input-12-43f817ab13c1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_summarized["title_2"][i] = simple_similarity_top_pairs[simple_similarity_top_pairs["title_1"] == row["title_1"]]["title_2"].values[0]
<ipython-input-12-43f817ab13c1>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_summarized["predicted_y"][i] = 1
<ipython-input-12-43f817ab13c1>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_summarized["reason"]

,title_1,title_2,similarity,predicted_y,reason
0,进修堂文集诗集,静春堂诗集,0.8661,1,inner predict
1,马书奎稿,苑洛集,0.9374,1,inner predict
2,游莱三体诗稿,燕堂诗稿,0.9521,1,first predict
3,鲍觉生诗钞,岭云海日楼诗钞,0.8824,1,inner predict
4,赋则,琴赋,,1,simple similarity


In [15]:
book_category_match = pd.read_excel("/content/book_category_match.xlsx")
df_summarized["category"] = ""

df_summarized.head()
for i, row in df_summarized.iterrows():
    try:
        df_summarized["category"][i] = book_category_match[book_category_match["title"] == row["title_2"]]["category"].values[0]
    except:
        pass
print(df_summarized.head())

# This output is for advanced users
df_summarized.to_excel("book_category_match_output.xlsx", index=False)

df_summarized = df_summarized.drop(columns=["title_2"])
df_summarized = df_summarized.rename(columns={"title_1": "title", "similarity": "score"})
print(df_summarized.head())
now = datetime.datetime.now()
now = now.strftime("%Y-%m-%d %H:%M:%S")

output_filename = f"book_category_match_output_{now}.xlsx"

df_summarized.to_excel(output_filename, index=False)

<ipython-input-15-598148db472a>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_summarized["category"][i] = book_category_match[book_category_match["title"] == row["title_2"]]["category"].values[0]


   title_1  title_2 similarity  predicted_y             reason category
0  进修堂文集诗集    静春堂诗集     0.8661            1      inner predict  集藏\四库别集
1     马书奎稿      苑洛集     0.9374            1      inner predict  集藏\四库别集
2   游莱三体诗稿     燕堂诗稿     0.9521            1      first predict  集藏\四库别集
3    鲍觉生诗钞  岭云海日楼诗钞     0.8824            1      inner predict  集藏\四库别集
4       赋则       琴赋                       1  simple similarity    艺藏\音乐
     title   score  predicted_y             reason category
0  进修堂文集诗集  0.8661            1      inner predict  集藏\四库别集
1     马书奎稿  0.9374            1      inner predict  集藏\四库别集
2   游莱三体诗稿  0.9521            1      first predict  集藏\四库别集
3    鲍觉生诗钞  0.8824            1      inner predict  集藏\四库别集
4       赋则                    1  simple similarity    艺藏\音乐


In [16]:
from google.colab import files

files.download(f'/content/{output_filename}')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>